# Testing Architectures of the Neural Networks

Using the refactored code within the dnn/models/ directory, we want to go through each model and ensure that the models meet certain standards.

*Consider replacing with unit testing*

In [1]:
import sys
sys.path.append('../../dnn/')
from model.nets.ieegcnn import iEEGCNN
from model.nets.ieegseq import iEEGSeq
import model.train
from model.train import traincnn, trainseq

import processing.util as util

import time
import numpy as np
# np.random.seed(1234)
from functools import reduce
import math as m
import os

# import DNN frameworks
import tensorflow as tf
import keras

# import high level optimizers, models and layers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding

# utility for datasets and training
# from keras.utils.training_utils import multi_gpu_model
from keras.datasets import imdb
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
# imports tensorflow
# from keras import backend as K

from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


%load_ext autoreload
%autoreload 2

/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imsize=32
n_colors =3 
num_classes=2
modeldim=2
DROPOUT=True

cnn = iEEGCNN(imsize=imsize,
              n_colors=n_colors, 
              num_classes=num_classes, 
              modeldim=modeldim, 
              DROPOUT=DROPOUT)
cnn.buildmodel()
cnn.summaryinfo()
print(cnn.model.input_shape)
print(cnn.model.summary())

{'DROPOUT': True,
 'filteringsize': (3, 3),
 'imsize': 32,
 'numchans': 3,
 'numclasses': 2,
 'numfilters': 32,
 'numlayers': (4, 2, 1),
 'poolingsize': (2, 2),
 'w_init': None}
(None, 32, 32, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2

# Show Code For Training Setup of CNN

In [3]:
dnnmodel = cnn.model
batch_size = 32
NUM_EPOCHS = 100
AUGMENT = True
cnn_trainer = traincnn.TrainCNN(dnnmodel, batch_size, NUM_EPOCHS, AUGMENT)

In [4]:
cnn_trainer.summaryinfo()

{'augment': True, 'batch_size': 32, 'epochs': 100}


In [11]:
# configure and initialize generator
tempdatadir = './test'
pattraindir = './test'
# list of patients to train on
listofpats_train = [
                'id001',
                'id002', 
                'id008', 
                'id010', 
                'id011', 
                'id012', 
                'id013'
            ]

cnn_trainer.configure(tempdatadir)
cnn_trainer.loadgenerator()
cnn_trainer.loaddirofdata(pattraindir, listofpats_train, LOAD=True)
# cnn_trainer.loaddata(datafile, imsize, n_colors)

In [15]:
testlist = ['hi', 'bye', 'dog']

test = ['hiseq.py', 'dabg', 'byeseq', 'cat', 'catdog']

for item in test:
    if any(x in item for x in testlist):
        print(item)

hiseq.py
byeseq
catdog


# Show Code For Training Setup of CNN-LSTM

In [55]:
finaldir = '/Volumes/ADAM LI/pydata/dnn/output/train_v6/'
weightsfile = os.path.join(finaldir, 'final_weights.h5')
modelfile = os.path.join(finaldir, '2dcnn_model.json')

# initialize loss function, SGD optimizer and metrics
loss = 'binary_crossentropy'
optimizer = keras.optimizers.Adam(lr=1e-4, 
                                beta_1=0.9, 
                                beta_2=0.999,
                                epsilon=1e-08,
                                decay=0.0)
metrics = ['accuracy']

# load json and create model
json_file = open(modelfile, 'r')
loaded_model_json = json_file.read()
json_file.close()

# fixed_cnn_model = ieegdnn.load_model(weightsfile, freeze=True)
fixed_cnn_model = keras.models.model_from_json(loaded_model_json)
fixed_cnn_model.load_weights(weightsfile)

# remove the last 2 dense FC layers and freeze it
fixed_cnn_model.pop()
fixed_cnn_model.pop()
fixed_cnn_model.pop()
fixed_cnn_model.pop()
# fixed_cnn_model.pop()
fixed_cnn_model.trainable = False
# modelconfig = fixed_cnn_model.compile(loss=loss, 
#                                         optimizer=optimizer,
#                                         metrics=metrics)
# fixed_cnn_model.add(keras.layers.Flatten())
# fixed_cnn_model.add(keras.layers.Dense(units=10000, activation='relu'))

print(fixed_cnn_model.summary())
print(fixed_cnn_model.output_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        1184      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 10, 64)        18496     
__________

In [44]:
imsize=32
n_colors =4
num_classes=2
modeldim=2
DROPOUT=True

# cnn = iEEGCNN(imsize=imsize,
#               n_colors=n_colors, 
#               num_classes=num_classes, 
#               modeldim=modeldim, 
#               DROPOUT=DROPOUT)
# cnn.buildmodel()
# cnn.summaryinfo()

# print("EACH CNN MODEL INPUT IS: ", cnn.model.input_shape)
# cnn.model.trainable = False
# cnn.configure()
# fixed_cnn_model = cnn.model

name = 'MIX'
name = 'SAME'
num_classes=2
num_timewins=5
DROPOUT=True
BIDIRECT=False
cnnseq = iEEGSeq(name=name,
                num_classes=num_classes,
                num_timewins=num_timewins,
                DROPOUT=DROPOUT,
                BIDIRECT=BIDIRECT)
cnnseq.buildmodel(fixed_cnn_model)
cnnseq.buildoutput()
config = cnnseq.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy']) 
convtest = cnnseq.model
print(convtest.summary())

# cnnseq.model.trainable_weights = False
# cnnseq.configure()
# cnnseq.model.summary()


ValueError: Input 0 is incompatible with layer flatten_21: expected min_ndim=3, found ndim=2

In [56]:
from keras import Model
from keras.layers import TimeDistributed, Dense, Dropout, Flatten, LSTM
size_mem = 100
size_fc = 1000
# create sequential model to get this all before the LSTM
currmodel = keras.models.Sequential()
currmodel.add(TimeDistributed(fixed_cnn_model, input_shape=(num_timewins, imsize, imsize, n_colors)))
currmodel.add(LSTM(units=size_mem, 
            activation='relu', 
            return_sequences=False))
# currmodel.add(Flatten())
currmodel.add(Dense(size_fc, activation='relu'))
currmodel.add(Dense(num_classes, activation='softmax'))

currmodel = Model(inputs=currmodel.input, outputs = currmodel.output)
config = currmodel.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy']) 
print(currmodel.summary())
print(currmodel.input_shape)
print(currmodel.output_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_14_input (I (None, 5, 32, 32, 4)      0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 5, 128)            158208    
_________________________________________________________________
lstm_13 (LSTM)               (None, 100)               91600     
_________________________________________________________________
dense_33 (Dense)             (None, 1000)              101000    
_________________________________________________________________
dense_34 (Dense)             (None, 2)                 2002      
Total params: 352,810
Trainable params: 352,810
Non-trainable params: 0
_________________________________________________________________
None
(None, 5, 32, 32, 4)
(None, 2)
